<a href="https://colab.research.google.com/github/manishiitg/ML_Experiments/blob/master/pytorch/101/pytorch_basics_LR_on_MNIST_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mainly experiment with pytorch dataset package and see how to load/train/evaluate a simple NN

In [0]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

import torch
import torch.optim as optim
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [5]:
# https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/logistic_regression/main.py#L33-L34
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Hyper-parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

train_dataset = torchvision.datasets.MNIST(root="mnist", train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)


test_dataset = torchvision.datasets.MNIST(root='mnist', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# Logistic regression model
model = nn.Linear(input_size, num_classes)


# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

# Train the model
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, 28*28)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Epoch [1/5], Step [100/600], Loss: 2.2120
Epoch [1/5], Step [200/600], Loss: 2.1136
Epoch [1/5], Step [300/600], Loss: 2.0921
Epoch [1/5], Step [400/600], Loss: 1.9398
Epoch [1/5], Step [500/600], Loss: 1.8558
Epoch [1/5], Step [600/600], Loss: 1.8179
Epoch [2/5], Step [100/600], Loss: 1.7063
Epoch [2/5], Step [200/600], Loss: 1.6512
Epoch [2/5], Step [300/600], Loss: 1.6446
Epoch [2/5], Step [400/600], Loss: 1.5749
Epoch [2/5], Step [500/600], Loss: 1.4473
Epoch [2/5], Step [600/600], Loss: 1.5682
Epoch [3/5], Step [100/600], Loss: 1.4511
Epoch [3/5], Step [200/600], Loss: 1.4504
Epoch [3/5], Step [300/600], Loss: 1.4043
Epoch [3/5], Step [400/600], Loss: 1.3293
Epoch [3/5], Step [500/600], Loss: 1.2701
Epoch [3/5], Step [600/600], Loss: 1.2817
Epoch [4/5], Step [100/600], Loss: 1.2880
Epoch [4/5], Step [200/600], Loss: 1.3772
Epoch [4/5], Step [300/600], Loss: 1.1883
Epoch [4/5], Step [400/600], Loss: 1.1049
Epoch [4/5], Step [500/600], Loss: 1.1122
Epoch [4/5], Step [600/600], Loss:

Let's build a NN for the same

In [6]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


Epoch [1/5], Step [100/600], Loss: 0.4277
Epoch [1/5], Step [200/600], Loss: 0.1985
Epoch [1/5], Step [300/600], Loss: 0.1048
Epoch [1/5], Step [400/600], Loss: 0.1146
Epoch [1/5], Step [500/600], Loss: 0.2991
Epoch [1/5], Step [600/600], Loss: 0.1882
Epoch [2/5], Step [100/600], Loss: 0.0633
Epoch [2/5], Step [200/600], Loss: 0.1760
Epoch [2/5], Step [300/600], Loss: 0.1249
Epoch [2/5], Step [400/600], Loss: 0.1007
Epoch [2/5], Step [500/600], Loss: 0.0770
Epoch [2/5], Step [600/600], Loss: 0.0645
Epoch [3/5], Step [100/600], Loss: 0.0299
Epoch [3/5], Step [200/600], Loss: 0.0286
Epoch [3/5], Step [300/600], Loss: 0.1951
Epoch [3/5], Step [400/600], Loss: 0.0309
Epoch [3/5], Step [500/600], Loss: 0.0947
Epoch [3/5], Step [600/600], Loss: 0.1202
Epoch [4/5], Step [100/600], Loss: 0.0631
Epoch [4/5], Step [200/600], Loss: 0.1022
Epoch [4/5], Step [300/600], Loss: 0.0399
Epoch [4/5], Step [400/600], Loss: 0.0950
Epoch [4/5], Step [500/600], Loss: 0.0561
Epoch [4/5], Step [600/600], Loss: